## Описание проекта

Оператор мобильной связи выяснил: многие клиенты пользуются архивными тарифами. Оператор хочет построить систему, способную проанализировать поведение клиентов и предложить пользователям один из двух новых тарифов.

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий новый тариф. Предобработка данных уже произведена.

Задача - построить модель с максимально большим значением accuracy. Долю правильных ответов должна быть не менее 0.75. Необходима проверка accuracy на тестовой выборке.

## Инструкция по выполнению проекта

* 		Откройте файл с данными и изучите его. Путь к файлу: datasets/users_behavior.csv. Скачать датасет
* 		Разделите исходные данные на обучающую, валидационную и тестовую выборки.
* 		Исследуйте качество разных моделей, меняя гиперпараметры. Кратко напишите выводы исследования.
* 		Проверьте качество модели на тестовой выборке.
* 		Дополнительное задание: проверьте модели на вменяемость.

## Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

* 		`сalls` — количество звонков,
* 		`minutes` — суммарная длительность звонков в минутах,
* 		`messages` — количество sms-сообщений,
* 		`mb_used` — израсходованный интернет-трафик в Мб,
* 		`is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).


## 1. Откройте и изучите файл

In [82]:
# Импортируем необходимые библиотеки
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

# Читаем исходный файл
client_data = pd.read_csv('/datasets/users_behavior.csv')

# Смотрим общую информацию
print('INFO')
display(client_data.info())
print('HEAD 10')
display(client_data.head(10))
print('DESCRIBE')
display(client_data.describe())

INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

HEAD 10


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


DESCRIBE


,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Учитывая, что нашим целевым признаком является значение `is_ultra` (т.е. является ли клиент пользователем тарифа "Ультра" (1) или тарифа "Смарт" (0)) - оценим объем этих групп клиентов.

In [83]:
client_data.groupby('is_ultra')['calls'].count().to_frame('users_number')

,users_number
is_ultra,
0,2229
1,985


Несмотря на то, что в условиях проекта укащано, что данные уже предобработаны, проведем контрольную проверку на пропуски и дубликаты:

In [84]:
client_data.isnull().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [85]:
client_data.duplicated().sum()

0

## 1. Общая информация. Выводы.

- исходный датафрейм представлен 3 214 объектами и 5 признаками
- наименования столбцов достаточно информативны и стандартизированы
- форматы данных в столбцах соответствуют содержанию и предстоящим вычислениям
- пропусков и дубликатов не обнаружено
- пользователи тарифа "Ультра" составляют 30% выборки, пользователи тарифа "Смарт" - 70%

## 2. Разбейте данные на выборки

В связи с отсутствием тестовой выборки, проведем разбиение исходных данные на обучающую, валидационную и тестовую выборки. В связи с достаточно небольшим объемом исходных данных (всего  3 214 записей) разумным представляется выделить основную часть датасета (70%) на обучающую выборку и по 15% на тестовую и валидационную выборки.

In [86]:
# Проведем последовательное разбиение. Сначала разделим датасет на обучающую и валидационную выборки (70% отдаем обучающей)
data_train, data_valid = train_test_split(client_data, test_size = 0.3, random_state = 12345)

# Теперь поделим пополам валидационную выборку, тем самым создав тестовую
data_valid, data_test = train_test_split(data_valid, test_size = 0.5, random_state=12345)

# Контрольная проверка размерности выборок
print('Размерность обучащей выборки:', data_train.shape)
print('Размерность валидационной выборки:', data_valid.shape)
print('Размерность тестовой выборки:', data_test.shape)

Размерность обучащей выборки: (2249, 5)
Размерность валидационной выборки: (482, 5)
Размерность тестовой выборки: (483, 5)


## 2. Разбивка данных на выборки. Выводы.

- исходные данные были разделены на три отдельных выборки:
    - обучающая (`data_train`), 70% всех данных
    - валидационная (`data_valid`), 15% всех данных
    - тестовая (`data_test`), 15% всех данных

## 3. Исследуйте модели

Исследуем качество разных моделей классификации (решающее дерево, случайный лес, логистическая регрессия), меняя гиперпараметры.

In [87]:
# Подготовим переменные для признаков и целевого признака различных выборок
features_train = data_train.drop(['is_ultra'], axis = 1)
target_train = data_train['is_ultra']
features_valid = data_valid.drop(['is_ultra'], axis = 1)
target_valid = data_valid['is_ultra']
features_test = data_test.drop(['is_ultra'], axis = 1)
target_test = data_test['is_ultra']

#### 3.1 Решающее дерево (DecisionTreeClassifier)

In [88]:
# Посмотрим на качество модели в зависимости от используемой глубины
for depth in range(1, 16):
    model_tree = DecisionTreeClassifier(max_depth = depth, random_state = 12345)
    model_tree.fit(features_train, target_train)
    predictions_tree_valid = model_tree.predict(features_valid)
    print('Tree_max_depth =', depth, ':', accuracy_score(target_valid,predictions_tree_valid))

Tree_max_depth = 1 : 0.7510373443983402
Tree_max_depth = 2 : 0.7800829875518672
Tree_max_depth = 3 : 0.7863070539419087
Tree_max_depth = 4 : 0.7883817427385892
Tree_max_depth = 5 : 0.7614107883817427
Tree_max_depth = 6 : 0.7842323651452282
Tree_max_depth = 7 : 0.7821576763485477
Tree_max_depth = 8 : 0.7821576763485477
Tree_max_depth = 9 : 0.7780082987551867
Tree_max_depth = 10 : 0.7883817427385892
Tree_max_depth = 11 : 0.7904564315352697
Tree_max_depth = 12 : 0.7946058091286307
Tree_max_depth = 13 : 0.7821576763485477
Tree_max_depth = 14 : 0.7821576763485477
Tree_max_depth = 15 : 0.7593360995850622


Максимальное качество модели (0.7946) достигается при `max_depth` = 11

#### 3.2 Случайный лес (RandomForestClassifier)

In [89]:
# Посмотрим на качество модели в зависимости от количества деревьев и используемой глубины
for estim in range(10,51,10):
    for depth in range (1, 11):
        model_forest = RandomForestClassifier(n_estimators = estim, max_depth = depth, random_state = 12345)
        model_forest.fit(features_train, target_train)
        predictions_forest_valid = model_forest.predict(features_valid)
        print('Forest_n_estimators =', estim,'and depth =', depth, ':', accuracy_score(target_valid,predictions_forest_valid))

Forest_n_estimators = 10 and depth = 1 : 0.7572614107883817
Forest_n_estimators = 10 and depth = 2 : 0.7614107883817427
Forest_n_estimators = 10 and depth = 3 : 0.7863070539419087
Forest_n_estimators = 10 and depth = 4 : 0.7883817427385892
Forest_n_estimators = 10 and depth = 5 : 0.7904564315352697
Forest_n_estimators = 10 and depth = 6 : 0.7966804979253111
Forest_n_estimators = 10 and depth = 7 : 0.7987551867219918
Forest_n_estimators = 10 and depth = 8 : 0.7883817427385892
Forest_n_estimators = 10 and depth = 9 : 0.7863070539419087
Forest_n_estimators = 10 and depth = 10 : 0.7966804979253111
Forest_n_estimators = 20 and depth = 1 : 0.7510373443983402
Forest_n_estimators = 20 and depth = 2 : 0.7925311203319502
Forest_n_estimators = 20 and depth = 3 : 0.7904564315352697
Forest_n_estimators = 20 and depth = 4 : 0.7966804979253111
Forest_n_estimators = 20 and depth = 5 : 0.7946058091286307
Forest_n_estimators = 20 and depth = 6 : 0.7966804979253111
Forest_n_estimators = 20 and depth = 7 

Максимальное качество (0.81327) достигается при `n_estimators` = 30 и `max_depth` = 10

#### 3.3 Логистическая регрессия (LogisticRegression)

In [90]:
model_log_reg = LogisticRegression(random_state=12345)
model_log_reg.fit(features_train, target_train)
print('Log_Reg_качество =', model_log_reg.score(features_valid,target_valid))

Log_Reg_качество = 0.6950207468879668


## 3. Исследование моделей. Выводы.

По результатам исследования моделей классификации получены следующие показатели качества:

- `дерево решений` - 0.7946, достигается при `max_depth` = 11
- `случайный лес` - 0.8133, достигается при `n_estimators` = 30 и `max_depth` = 10
- `логистическая регрессия` - 0.6950

## 4. Проверьте модель на тестовой выборке

Наиболее качественной является модель `случайный лес`. Проверим ее на тестовой выборке.

In [91]:
model = RandomForestClassifier(n_estimators = 30, max_depth = 10, random_state = 12345)
model.fit(features_train, target_train)
predictions_forest_test = model.predict(features_test)
accuracy_score(target_test,predictions_forest_test)

0.8115942028985508

## 4. Проверка модели на тестовой выборке. Выводы.

На тестовой выборке получили значение `accuracy` = 0.8116, что очень близко к значению, полученному на обучающей выборке (0.8133).
Таким образом, можно утвержать, что нам удалось избежать проблемы переобучения модели. Протестированная нами модель может, с достаточно высокой эффективностью, решать задачу бинарной классификации клиентов Оператора. 

## 5. (бонус) Проверьте модели на адекватность

Удостоверимся в адекватности нашей модели используя фиктивный классификатор `DummyClassifire`

In [92]:
from sklearn.dummy import DummyClassifier

In [93]:
# настраиваем модель
dummy_model = DummyClassifier(strategy="uniform")
# обучаем
dummy_model.fit(features_train, target_train)
# оцениваем качество
print ('DummyClassifier_качество на обучающей выборке:', dummy_model.score(features_train, target_train))
# сравним с тестовой выборкой
dummy_predictions_test = dummy_model.predict(features_test)
print ('DummyClassifier_качество на тестовой выборке:', accuracy_score(target_test, dummy_predictions_test))

DummyClassifier_качество на обучающей выборке: 0.4926634059582036
DummyClassifier_качество на тестовой выборке: 0.5238095238095238


## Общий вывод.

#### Цель исследования

На основании данных о поведении клиентов, которые уже перешли на тарифы Smart и Ultra, необходимо построить модель классификации с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75.

#### Общая информация о данных

- исходный датафрейм представлен 3 214 объектами и 5 признаками
- наименования столбцов достаточно информативны и стандартизированы
- форматы данных в столбцах соответствуют содержанию и предстоящим вычислениям
- пропусков и дубликатов не обнаружено
- пользователи тарифа "Ультра" составляют 30% выборки, пользователи тарифа "Смарт" - 70%

#### Результаты исследования различных моделей

Для обучения были применены следующие алгоритмы классификации:

- `решающее дерево`. Алгоритм DecisionTreeClassifier. Максимальное значение accuracy = 0.7946 (при max_depth = 11)
- `случайный лес`. Алгоритм RandomForestClassifier. Максимальное значение accuracy = 0.8133 (при n_estimators = 30 и max_depth = 10). В связи с наибольшим значением `accuracy`, среди всех протестированных моделей, ттот алгоритм выбран для предсказаний в тестовой выборке.
- `логистическая регрессия`. Алгоритм LogisticRegressior. Значение accuracy = 0.6950

Применение обученного алгоритма `случайный лес` на тестовой выборке показало значение accuracy = 0.8116, что, в свою очередь, подтвердило эффективность выбранной модели.

Также, была осуществлена оценка модели на адекватность при помощи алгоритма `DummyClassifier`. На тестовой выборке применение этого алгортима показало значение accuracy = 0.4969, что подтверждает адекватность выбранной для предсказания модели.